## Encoding of sd-shell $H_Q$ for quantum annealing

In [ ]:
## 

## Encoding of sd shell $H$ for quantum annealing

Imports

In [2]:
from src.hamiltonian_utils import FermiHubbardHamiltonian
from src.nuclear_physics_utils import get_twobody_nuclearshell_model,SingleParticleState,QuadrupoleOperator,J2operator
import numpy as np
import torch
from typing import Dict
import scipy
from src.qml_models import AdaptVQEFermiHubbard
from src.qml_utils.train import Fit
from src.qml_utils.utils import configuration
from scipy.sparse.linalg import eigsh,expm_multiply
from tqdm import trange
import matplotlib.pyplot as plt
from src.utils_quasiparticle_approximation import QuasiParticlesConverterOnlynnpp,HardcoreBosonsBasis
from scipy.sparse import lil_matrix


file_name='data/usdb.nat'

SPS=SingleParticleState(file_name=file_name)



nparts=[(2,0)]
titles=[r'$^{18}$O']



size_a=SPS.energies.shape[0]//2
size_b=SPS.energies.shape[0]//2

title=r'$^{18}$O'

Two-body matrix

In [3]:
twobody_matrix,energies=get_twobody_nuclearshell_model(file_name=file_name)

Computing the matrix, pls wait... (u_u) 



100%|██████████| 24/24 [00:54<00:00,  2.29s/it]


Nuclear Shell Model Hamiltonian

In [4]:
from scipy.sparse import identity
#Quadrupole Operator
energy_errors=[]
fidelities=[]
for idx,npart in enumerate(nparts):
    nparticles_a,nparticles_b=npart
    title=titles[idx]
    
    # compute the NSM Hamiltonian
    NSMHamiltonian=FermiHubbardHamiltonian(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,symmetries=[SPS.total_M_zero])
    print('size=',size_a+size_b,size_b)
    NSMHamiltonian.get_external_potential(external_potential=SPS.energies[:size_a+size_b])
    if file_name=='data/cki':
        twobody_matrix,energies=get_twobody_nuclearshell_model(file_name=file_name)

        NSMHamiltonian.get_twobody_interaction(twobody_dict=twobody_matrix)
    else:
        NSMHamiltonian.twobody_operator=scipy.sparse.load_npz(f'data/nuclear_twobody_matrix/usdb_{nparticles_a}_{nparticles_b}.npz')
    NSMHamiltonian.get_hamiltonian()

    
    shifted_hamiltonian=NSMHamiltonian.hamiltonian


    egs,psi0=eigsh(shifted_hamiltonian,k=1,which='SA')

    print('ground state energy=',egs[0])

size= 24 12
ground state energy= -11.931788251561516


In [ ]:
from qiskit_nature.second_q.hamiltonians import ElectronicEnergy
from qiskit_nature.second_q.operators import FermionicOp
import numpy as np

# Define number of spin-orbitals (say, N)
N = size_a  # for example



v_matrix=np.zeros((N, N, N, N), dtype=np.float64)
for key, value in twobody_matrix.items():
    a,b,c,d=key
    
    if a<size_a and b<size_a and c<size_a and d<size_a:
        v_matrix[a, b, c, d] = value



hamiltonian = ElectronicEnergy.from_raw_integrals(SPS.energies, v_matrix)
fermionic_op = hamiltonian.second_q_op()

In [ ]:
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.mappers import JordanWignerMapper

# Start building terms
hamiltonian_terms = {}
n_spin_orbitals =  size_a  # Assuming spin-orbitals are double the single-particle states
# One-body terms: e_a * c†_a c_a
for a, e_a in enumerate(SPS.energies[:size_a]):
    label = f"+_{a} -_{a}"
    hamiltonian_terms[label] = hamiltonian_terms.get(label, 0) + e_a

# Two-body terms: (1/4) * v_abcd * c†_a c†_b c_d c_c
for (a, b, c, d), v_abcd in twobody_matrix.items():
    if a<size_a and b<size_a and c<size_a and d<size_a:
        label = f"+_{a} +_{b} -_{d} -_{c}"
        coef = 0.25 * v_abcd
        hamiltonian_terms[label] = hamiltonian_terms.get(label, 0) + coef

# Create FermionicOp
ferm_op = FermionicOp(hamiltonian_terms, num_spin_orbitals=n_spin_orbitals)

# (Optional) Print to inspect
print(ferm_op)

In [8]:
mapper = JordanWignerMapper()
qubit_op = mapper.map(ferm_op)

In [ ]:
print(qubit_op)
from scipy.sparse import csr_matrix
n_qubits = qubit_op.num_qubits

for idx_b,b in enumerate(NSMHamiltonian.basis):
    for idx_d,d in enumerate(NSMHamiltonian.basis):
        b_1,b_2=np.nonzero(b)[0]
        d_1,d_2=np.nonzero(d)[0]
        print(f'i={(idx_b)},j={(idx_d)} ->{csr_matrix(qubit_op.to_matrix())[2**(b_1-1)+2**(b_2-1),2**(d_1-1)+2**(d_2-1)]} \n')
for i in range(1,n_qubits+1):
    for j in range(1,n_qubits+1):
        
        print(f'i={(i-1)},j={(j-1)} ->{csr_matrix(qubit_op.to_matrix())[2**(i-1),2**(j-1)]} \n')

Driver Hamiltonian